In [4]:
import os
import time
import json
import re
from dotenv import load_dotenv
import pandas as pd
import google.generativeai as genai

### **Carrega as variáveis de ambiente**

In [5]:
load_dotenv()

True

### **Configura e inicializa o modelo**

In [6]:
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel('gemini-2.0-flash')

### **Carrega os arquivos de entrada**

In [7]:
input_path = 'data/conversas_leads.csv'
input = pd.read_csv(input_path)

### **Extrai as informações da entrada**

In [ ]:
def extrair_informacoes(conversa):
    prompt = f"""
    Extraia as seguintes informações da conversa e retorne em formato JSON:

    {{
        "nome": "Nome do Lead",
        "email": "Email do Lead",
        "telefone": "Telefone do Lead",
        "cidade": "Cidade de interesse",
        "regiao": "Região de interesse",
        "tipo_imovel": "Tipo de imóvel (apartamento, casa, terreno, etc.)",
        "quartos": "Número de quartos",
        "orcamento": "Orçamento",
        "caracteristicas": "Características adicionais (piscina, vaga de garagem, etc.)",
        "disponibilidade": "Disponibilidade para visita",
        "intencao": "Intenção (compra, aluguel, investimento, etc.)"
    }}

    Conversa:
    {conversa}

    Informações extraídas:
    """
    response = model.generate_content(prompt)
    try:
        # Adequa a resposta ao formato json
        json_str = re.sub(r'```json\n', '', response.text)
        json_str = re.sub(r'\n```', '', json_str)
        # Decodifica o JSON
        dados = json.loads(json_str)
        return dados
    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar JSON: {response.text}")
        print(f"Erro específico: {e}")
        return None

### **Processa os dados de entrada**

In [9]:
def processar_csv():
    df = input
    resultados = []
    for index, row in df.iterrows():
        conversa = row['conversa']
        informacoes = extrair_informacoes(conversa)
        if informacoes:
            resultados.append(informacoes)
        time.sleep(1)  # Atraso de 1 segundo entre as requisições
    return resultados

### **Salva o resultado na pasta de resultados**

In [10]:
def salvar_resultados_json(resultados, caminho_json):
    with open(caminho_json, 'w', encoding='utf-8') as f:
        json.dump(resultados, f, ensure_ascii=False, indent=4)

### **Executa o código**

In [ ]:
resultados = processar_csv()

In [ ]:
path_output = 'results/resultados.json'
salvar_resultados_json(resultados, path_output)